In [1]:
import numpy as np
from scipy.integrate import solve_ivp,odeint
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from collections import namedtuple

In [2]:
use_matplotlib = False
use_vispy = True
use_script_rendering = True

In [3]:

# Double Pendulum Model
# https://matplotlib.org/stable/gallery/animation/double_pendulum.html
from numpy import sin, cos

G = 9.8  # acceleration due to gravity, in m/s^2
L1 = 1.0  # length of pendulum 1 in m
L2 = 1.0  # length of pendulum 2 in m
L = L1 + L2  # maximal length of the combined pendulum
M1 = 2.0  # mass of pendulum 1 in kg
M2 = 5.0  # mass of pendulum 2 in kg
t_stop = 2.5  # how many seconds to simulate
history_len = 500  # how many trajectory points to display

def derivs(t, state):
    dydx = np.zeros_like(state)
    dydx[0] = state[1]

    delta = state[2] - state[0]
    den1 = (M1+M2) * L1 - M2 * L1 * cos(delta) * cos(delta)
    dydx[1] = ((M2 * L1 * state[1] * state[1] * sin(delta) * cos(delta)+ M2 * G * sin(state[2]) * cos(delta)
                + M2 * L2 * state[3] * state[3] * sin(delta)- (M1+M2) * G * sin(state[0]))/ den1)

    dydx[2] = state[3]

    den2 = (L2/L1) * den1
    dydx[3] = ((- M2 * L2 * state[3] * state[3] * sin(delta) * cos(delta)+ (M1+M2) * G * sin(state[0]) * cos(delta)
                - (M1+M2) * L1 * state[1] * state[1] * sin(delta)- (M1+M2) * G * sin(state[2]))/ den2)

    return dydx


#v_derivs = np.vectorize(derivs,signature='(),(4)->(4)')
#state= np.array([[1,2,3,4],[5,6,7,8]])
#v_derivs(1,state)

In [4]:
import random
#animate a system of 100 double pendulums
n_sys = 100000
# initial state using random initial conditions
p1,s1=2,0.0001
p2,s2=-1.8,0.0001
p10=np.random.normal(loc=p1,scale=s1,size=n_sys)
p20=np.random.normal(loc=p2,scale=s2,size=n_sys)
init_state=np.array([p10,p10 - p1,p20,p20 - p2]).T
u0=init_state.flatten()

#v_derivs(1,init_state).shape

In [5]:
def multi_dbp(t,states):
    # multi_dbp is a vectorized version of the derivs function to
    # handle multiple double pendulums at once
    states=states.reshape(-1,4)
    return derivs(t,states.T).T.flatten()#v_derivs(1,states).flatten()

u0=init_state.flatten() #initial states

In [6]:
from scipy.integrate import odeint
import os
t_end=10
steps=1000
t=np.linspace(0, t_end, steps)

#if there is a double_pendulum.npz file, load it else compute the trajectory
if os.path.exists('double_pendulum.npz'):
    data = np.load('double_pendulum.npz')
    yt = data['yt']
else:
    %time y = odeint(multi_dbp, u0, t,tfirst=True,ml=3, mu=3)
    yt=y.reshape(-1,n_sys,4)
    y=[] # to save memory
    yt.shape,yt.dtype


In [7]:
def pendulum_points(yt, i):
    pend_points = np.zeros((n_sys, 1, 2))
    pend_points = np.append(pend_points, (L1 * np.array([[sin(yt[i, :, 0])], [-cos(yt[i, :, 0])]])).transpose(2, 1, 0), axis=1)
    pend_points = np.append(pend_points, (L2 * np.array([[sin(yt[i, :, 2])], [-cos(yt[i, :, 2])]]) + L1 * np.array([[sin(yt[i, :, 0])], [-cos(yt[i, :, 0])]])).transpose(2, 1, 0), axis=1)
    return pend_points    

def pend_ss(yt, i ,l, wind=100):
    if i<=wind:
        ss=yt[0:i,:,2*(l-1):2*(l-1)+2].transpose(1,0,2)#(yt[0:i,:,2:4]).reshape(n_sys,i,-1, order= 'C')
    else:
        ss=yt[i-wind:i,:,2*(l-1):2*(l-1)+2].transpose(1,0,2)
    return ss
    

In [8]:
%matplotlib qt6
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.collections import LineCollection
import matplotlib
cols = plt.cm.jet(np.random.rand(n_sys))#np.random.rand(n_sys,4)
#matplotlib.rcParams['animation.ffmpeg_path'] = "C:/users/nandh/anaconda3/envs/Dynamics/Library/bin/ffmpeg.exe"


In [9]:
%matplotlib qt6
if use_matplotlib:
    
    #animate the result of all lines
    #make a 2d array of all x1, y1 , x2 and y2 values




    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.grid()
    #make axis limits
    ax.set_xlim(-L, L)
    ax.set_ylim(-L, L)

    #switch axis markers off
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    #plot all empty lines
    #lines = [ax.plot([], [], 'o-', lw=1)[0] for i in range(0,y.shape[1],4)]
    #set background color to black
    ax.set_facecolor('black')



    i=0
    pend_points = pendulum_points(yt, i)
    lines = LineCollection(pend_points, colors=cols, linestyle='solid', joinstyle='round', capstyle='round')
    ax.add_collection(lines)
    #make animate function for all double pendulums
    def animate(i):
        
        pend_points = pendulum_points(yt, i)
        lines.set_segments(pend_points)
        #ax.add_collection(lines)
        progress_callback(i, yt.shape[0])
        return [lines]

    #callback function to track progress
    def progress_callback(i, n):
        print(f"Progress: {i}/{n}",end='\r')
        #if i % (n // 1000) == 0:
        #    print(f"Progress: {i}/{n}")




    
    ani_dp = animation.FuncAnimation(fig, animate, yt.shape[0], interval=10, blit=True)
    #set ffmpeg path

    ani_dp.save('double_pendulum.mp4', writer='ffmpeg', fps=30, dpi=300)

In [10]:
if use_matplotlib:
    #animate the state space of the double pendulum second link

    fig, ax = plt.subplots()
    ax.set_xlim(-3,27)
    ax.set_ylim(-15,15)

    ax.grid()
    ax.set_facecolor('black')
    plt.title('State Space of second link of double pendulum')
    plt.xlabel('theta_2')
    plt.ylabel('omega_2')

    i=1
    wind=100
    if i<=wind:
        LCl2=yt[0:i,:,2:4].transpose(1,0,2)#(yt[0:i,:,2:4]).reshape(n_sys,i,-1, order= 'C')
    else:
        LCl2=yt[i-wind:i,:,2:4].transpose(1,0,2)


    lines=LineCollection(LCl2,colors=cols)

    ax.add_collection(lines)

    def progress_callback(i, n):
        print(f"Progress: {i}/{n}",end='\r')

    def animate(i):
        progress_callback(i, yt.shape[0])
        if i<=wind:
            LCl2=yt[0:i,:,2:4].transpose(1,0,2)#(yt[0:i,:,2:4]).reshape(n_sys,i,-1, order= 'C')
        else:
            LCl2=yt[i-wind:i,:,2:4].transpose(1,0,2)
        lines.set_segments(LCl2)
        #ax.add_collection(lines)

        return [lines]

    ani_sl = animation.FuncAnimation(fig, animate, yt.shape[0], interval=10, blit=True)
    ani_sl.save('double_pendulum_state_space_link2.mp4', writer='ffmpeg', fps=30, dpi=300)

In [11]:
if use_matplotlib:
    #animate the state space of the double pendulum first link

    fig, ax = plt.subplots()
    ax.set_xlim(-15,15)
    ax.set_ylim(-15,15)

    ax.grid()
    ax.set_facecolor('black')
    plt.title('State Space of second link of double pendulum')
    plt.xlabel('theta_2')
    plt.ylabel('omega_2')

    i=1
    wind=100
    if i<=wind:
        LCl2=yt[0:i,:,0:2].transpose(1,0,2)#(yt[0:i,:,2:4]).reshape(n_sys,i,-1, order= 'C')
    else:
        LCl2=yt[i-wind:i,:,0:2].transpose(1,0,2)


    lines=LineCollection(LCl2,colors=cols)

    ax.add_collection(lines)
    def progress_callback(i, n):
        print(f"Progress: {i}/{n}",end='\r')
    def animate(i):
        progress_callback(i, yt.shape[0])
        if i<=wind:
            LCl2=yt[0:i,:,0:2].transpose(1,0,2)#(yt[0:i,:,2:4]).reshape(n_sys,i,-1, order= 'C')
        else:
            LCl2=yt[i-wind:i,:,0:2].transpose(1,0,2)
        lines.set_segments(LCl2)
        #ax.add_collection(lines)

        return [lines]

    ani_fl = animation.FuncAnimation(fig, animate, yt.shape[0], interval=10, blit=True)
    ani_fl.save('double_pendulum_state_space_link1.mp4', writer='ffmpeg', fps=30, dpi=300)

In [12]:
import vispy
vispy.use('pyside6') #or pyqt5, pyqt6, pyside2, pyside6 if there is an error
from vispy import app, scene
from vispy.scene.visuals import create_visual_node
import imageio

In [13]:
def generate_connect(M):
    #if M is an array of shape (n, m,2/3) we use vectorisation to generate the connect array
    # else if M is a list of arrays we use a loop
    if isinstance(M, np.ndarray) and M.ndim == 3:
        n = M.shape[0]
        m = M.shape[1]

        # make continuous connect for first segment
        c1 = np.arange(0, m-1,dtype=int)
        c1 = np.column_stack((c1, c1 + 1))

        # repeat this for each segment and make a 2D array
        conn = np.repeat(c1[np.newaxis, :, :], n, axis=0)
        conn = conn + (np.arange(n) * m)[:,np.newaxis, np.newaxis]
        #flatten both connect and M
        conn = conn.reshape(-1, M.shape[2])
        M = M.reshape(-1, M.shape[2])
    elif isinstance(M, list):
        # if M is a list of arrays, we loop through each array and generate the connect array
        conn = []
        sc = 0
        for i, Mi in enumerate(M):
            if isinstance(M, list):
                Mi = np.array(Mi)
            m = Mi.shape[0]
            
            
            c1 = np.arange(0, m-1,dtype=int)
            #print(c1)
            c1 = np.column_stack((c1, c1 + 1))
            conn.append(c1 + sc)
            sc = sc + m
        conn = np.concatenate(conn, axis=0)
        M = np.concatenate(M, axis=0)
    else:
        raise ValueError("Input must be a 3D numpy array or a list of 2D arrays.")
    return conn,M



In [14]:
def generate_colors(colors,M):
    #repeat same color for each vertices 
    #colors has length of number of segments
    #returns with colors of length of number of vertices
    if isinstance(M, np.ndarray) :
        n = M.shape[0]
        m = M.shape[1]
        #print(n,m)
        # make continuous connect for first segment
        out =(np.repeat(colors[:,np.newaxis], m, axis=1))
        #print(out.shape)
        return out.reshape(-1, colors.shape[1])
    elif isinstance(M, list):
        # if M is a list of arrays, we loop through each array repeat same color for each segment
        colout = []
        for i, Mi in enumerate(M):
            if isinstance(M, list):
                Mi = np.array(Mi)
            m = Mi.shape[0]
            #print(m)
            colout.append(np.repeat(colors[i][np.newaxis, :], m, axis=0))
        return np.concatenate(colout, axis=0)
        

In [15]:
def black_plot(plot1):
    plot1.view.bgcolor = 'black'
    plot1.xaxis.axis.tick_color = 'white'
    plot1.yaxis.axis.tick_color = 'white'
    plot1.xaxis.axis.text_color = 'white'
    plot1.yaxis.axis.text_color = 'white'
    plot1.title._text_visual.color = 'white'
    return plot1


    

In [16]:
def pend_ssa(yt, i ,wind=100):
    if i<=wind:
        ss=yt[0:i,:,[1,3]].transpose(1,0,2)
    else:
        ss=yt[i-wind:i,:,[1,3]].transpose(1,0,2)
    return ss

In [17]:
if not use_script_rendering:
    import vispy.plot as vp

    # Create a canvas showing plot data
    fig = vp.Fig(bgcolor='black', size=(1200, 800),show=False)
    pos= pendulum_points(yt, 450)
    pos_ss = pend_ss(yt, 450,1)

    colors = generate_colors(cols, pos)
    connect,pos = generate_connect(pos)

    colors_ss = generate_colors(cols, pos_ss)
    connect_ss,pos_ss = generate_connect(pos_ss)

    pos_ssa = pend_ssa(yt, 450)

    colors_ssa = generate_colors(cols, pos_ssa)
    connect_ssa,pos_ssa = generate_connect(pos_ssa)

    pos_ss2 = pend_ss(yt, 450,2)
    colors_ss2 = generate_colors(cols, pos_ss2)
    connect_ss2,pos_ss2 = generate_connect(pos_ss2)

    plot1 = fig[0, 0]
    plot2 = fig[0, 1:3]
    plot3= fig[1, 0]
    plot4= fig[1, 1:3]

    #plot
    Line=plot1.plot(pos, connect=connect, color=colors, width=3, marker_size=0, title='Double Pendulum')
    plot1.xaxis.axis.axis_label = 'X'
    plot1.yaxis.axis.axis_label = 'Y'
    Line_ss1=plot2.plot(pos_ss, connect=connect_ss, color=colors_ss, width=3, marker_size=0,
                        title='State Space of First link of Double Pendulum')
    plot2.xaxis.axis.axis_label = 'theta_1'
    plot2.yaxis.axis.axis_label = 'omega_1'
    Line_ssa=plot3.plot(pos_ssa, connect=connect_ssa, color=colors_ssa, width=3, marker_size=0
                    ,title='Omega_1 vs Omega_2')
    plot3.xaxis.axis.axis_label = 'omega_1'
    plot3.yaxis.axis.axis_label = 'omega_2'
    Line_ss2=plot4.plot(pos_ss2, connect=connect_ss2, color=colors_ss2, width=3, marker_size=0
                        ,title='State Space of Second Link of Double Pendulum')
    plot4.xaxis.axis.axis_label = 'theta_2'
    plot4.yaxis.axis.axis_label = 'omega_2'


    plot1.view.camera.set_range(x=(-L, L), y=(-L, L))
    plot1.view.camera.aspect = 1
    plot2.view.camera.set_range(x=(-15, 15), y=(-15, 15))
    #plot2.view.camera.aspect = 1
    plot3.view.camera.set_range(x=(-15, 15), y=(-15, 15))
    #plot3.view.camera.aspect = 1
    plot4.view.camera.set_range(x=(-3, 27), y=(-15, 15))

    black_plot(plot1)
    black_plot(plot2)
    black_plot(plot3)
    black_plot(plot4)
    i=0
    colors_ss = generate_colors(cols, pos_ss)
    colors = generate_colors(cols, pos)
    colors_ssa = generate_colors(cols, pos_ssa)
    colors_ss2 = generate_colors(cols, pos_ss2)
    def update_plot(ev):
        global i, yt, L1, cols, colors_ss, colors, colors_ssa, colors_ss2
        wind=100
        i = (i + 1) % yt.shape[0]
        pos = pendulum_points(yt, i)
        pos_ss = pend_ss(yt, i,1)
        pos_ssa = pend_ssa(yt, i)
        pos_ss2 = pend_ss(yt, i,2)
        
        


        if True:#i <= wind + 2:
            colors_ss = generate_colors(cols, pos_ss)
            colors = generate_colors(cols, pos)
            colors_ssa = generate_colors(cols, pos_ssa)
            colors_ss2 = generate_colors(cols, pos_ss2)
        connect,pos = generate_connect(pos)
        connect_ss,pos_ss = generate_connect(pos_ss)
        connect_ssa,pos_ssa = generate_connect(pos_ssa)
        connect_ss2,pos_ss2 = generate_connect(pos_ss2)
        Line.set_data(pos, connect=connect, color=colors)
        Line_ss1.set_data(pos_ss, connect=connect_ss, color=colors_ss)
        Line_ssa.set_data(pos_ssa, connect=connect_ssa, color=colors_ssa)
        Line_ss2.set_data(pos_ss2, connect=connect_ss2, color=colors_ss2)
        #fig.scene.canvas.update()
        print(f"Progress: {i}/{yt.shape[0]}",end='\r')
        #if i > yt.shape[0] - 1:
        #    timer.stop()
        #    timer.disconnect()

    #timer = app.Timer(interval=0.01, connect=update_plot)



    ##if __name__ == '__main__':
        #$fig.app.run()
        #timer.start()

In [18]:
if use_script_rendering:
    #free RAM
    yt=[]
    
    import Vispy_anim
    Vispy_anim.main()
else:
    writer =imageio.get_writer("Doub_Pend_visp.mp4",fps=10,
                            format='ffmpeg', ffmpeg_params=['-b:v', '4000k','-s', '1200x800'])
    update_plot(None)
    fig.scene.canvas.update()
    fig.scene.canvas.app.process_events() 
    for i in range(yt.shape[0]-2):#yt.shape[0]
        update_plot(None)
        print(f"Progress: {i}/{yt.shape[0]}", end='\r')
        frame=fig.scene.canvas.render()
        writer.append_data(frame)
        del frame
        fig.scene.canvas.update()
        fig.scene.canvas.app.process_events() 
    writer.close()


(100000, 4)


TypeError: concatenate() got multiple values for argument 'axis'